In [ ]:
#!pip install surprise
#!pip install ace-tools

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_distances, cosine_similarity
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split


diet_df = pd.read_csv("/content/diet.csv")
recent_activity = pd.read_csv("/content/recent_activity.csv")
user_profiles_df = pd.read_csv("/content/user_Profiles.csv")

In [ ]:
diet_df.columns

Index(['Meal_Id', 'Name', 'catagory', 'description', 'Veg_Non', 'Nutrient',
       'Disease', 'Diet', 'Price'],
      dtype='object')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Load dataset
diet_df = pd.read_csv("/content/diet.csv")

# Handle missing values
diet_df.fillna("", inplace=True)

# Ensure consistent column names
diet_df.rename(columns={"catagory": "Category", "description": "Description"}, inplace=True)

# Combine relevant text columns for content-based filtering
diet_df["combined_text"] = (
    diet_df["Name"] + " " + diet_df["Category"] + " " + diet_df["Description"] +
    " " + diet_df["Nutrient"] + " " + diet_df["Disease"] + " " + diet_df["Diet"]
)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1,2), min_df=2)
tfidf_matrix = tfidf_vectorizer.fit_transform(diet_df["combined_text"])

# Compute Cosine Similarity Matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Mapping meal names to indices for lookup
indices = pd.Series(diet_df.index, index=diet_df["Name"]).drop_duplicates()

# Content-Based Recommendation Function
def content_based_recommendations(meal_name, top_n=5):
    if meal_name not in indices:
        return pd.DataFrame(columns=["Name", "Category", "Nutrient", "Diet"])

    idx = indices[meal_name]
    sim_scores = list(enumerate(cosine_sim[idx].flatten()))  # Ensure 1D list
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]  # Exclude itself

    # Extract valid indices within range
    meal_indices = [i[0] for i in sim_scores if i[0] < len(diet_df)]

    return diet_df.iloc[meal_indices][["Name", "Category", "Nutrient", "Diet"]]

# Example Usage:
sample_meal = diet_df["Name"].iloc[0]  # Pick the first meal as an example
recommended_meals = content_based_recommendations(sample_meal, top_n=5)

# Display Recommendations
print("Recommended Meals for:", sample_meal)
print(recommended_meals)


Recommended Meals for: summer squash salad
                       Name Category Nutrient  \
78      summer squash salad    salad    fiber   
187  prawn and litchi salad    salad    fiber   

                                                  Diet  
78    alkaline_diet low_fat_diet ketogenic_diet low...  
187   low_fat_diet ketogenic_diet low_sodium_diet h...  


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix

# Load dataset
recent_activity_df = pd.read_csv("/content/recent_activity.csv")

# Aggregate user interactions to ensure unique (User_Id, Meal_Id) pairs
recent_activity_agg = recent_activity_df.groupby(["User_Id", "Meal_Id"]).sum().reset_index()

# Pivot the data to create a User-Item interaction matrix
interaction_matrix = recent_activity_agg.pivot(index="User_Id", columns="Meal_Id", values="Liked").fillna(0)

# Convert to sparse matrix format
interaction_sparse = csr_matrix(interaction_matrix.values)

### **1. Improved Item-Based Filtering Using Adjusted Cosine Similarity**
# Normalize user interactions (Mean-Centered Ratings)
interaction_matrix_normalized = interaction_matrix.sub(interaction_matrix.mean(axis=1), axis=0).fillna(0)

# Compute Adjusted Cosine Similarity for Items
item_similarity_adjusted = cosine_similarity(interaction_matrix_normalized.T)  # Transpose for item-item similarity

# Convert to DataFrame
item_sim_df_adjusted = pd.DataFrame(item_similarity_adjusted, index=interaction_matrix.columns, columns=interaction_matrix.columns)

# Item-Based Recommendation Function
def item_based_recommendations_adjusted(meal_id, top_n=5):
    """
    Recommends similar meals based on item-based collaborative filtering using Adjusted Cosine Similarity.
    """
    if meal_id not in item_sim_df_adjusted.index:
        return pd.DataFrame(columns=["Meal_Id", "Adjusted_Cosine_Similarity"])

    # Get top-N most similar meals (excluding itself)
    similar_meals = item_sim_df_adjusted[meal_id].sort_values(ascending=False)[1:top_n+1]

    # Normalize similarity scores
    min_sim = similar_meals.min()
    max_sim = similar_meals.max()
    if max_sim - min_sim > 0:
        similar_meals = (similar_meals - min_sim) / (max_sim - min_sim)  # Normalize to [0,1]

    return pd.DataFrame(similar_meals).reset_index().rename(columns={meal_id: "Adjusted_Cosine_Similarity"})

### **2. User-Based Filtering Using Cosine Similarity**
# Compute User Similarity Matrix (Cosine Similarity)
user_similarity = cosine_similarity(interaction_sparse)

# Convert to DataFrame
user_sim_df = pd.DataFrame(user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)

# User-Based Recommendation Function
def user_based_recommendations_cosine(user_id, top_n=5):
    """
    Recommends meals for a user based on user-based collaborative filtering using cosine similarity.
    """
    if user_id not in user_sim_df.index:
        return pd.DataFrame(columns=["Meal_Id", "Predicted_Score"])

    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:top_n+1].index
    similar_users_ratings = interaction_matrix.loc[similar_users].mean(axis=0).sort_values(ascending=False)  # Average ratings from similar users

    return pd.DataFrame(similar_users_ratings.head(top_n)).reset_index().rename(columns={0: "Predicted_Score"})

# Example Usage: Item-Based Recommendations (Realistic)
sample_meal_id = recent_activity_df["Meal_Id"].iloc[0]  # First meal in dataset
recommended_items_item_based = item_based_recommendations_adjusted(sample_meal_id, top_n=5)

# Example Usage: User-Based Recommendations
sample_user_id = recent_activity_df["User_Id"].iloc[0]  # First user in dataset
recommended_items_user_based = user_based_recommendations_cosine(sample_user_id, top_n=5)

# Display both recommendations
print("🔥 **Realistic Item-Based Collaborative Filtering (Adjusted Cosine Similarity)**")
print(recommended_items_item_based)

print("\n🔥 **User-Based Collaborative Filtering (Cosine Similarity)**")
print(recommended_items_user_based)


🔥 **Realistic Item-Based Collaborative Filtering (Adjusted Cosine Similarity)**
      Meal_Id  Adjusted_Cosine_Similarity
0  meal_id263                         1.0
1  meal_id159                         1.0
2  meal_id248                         1.0
3  meal_id241                         1.0
4  meal_id130                         1.0

🔥 **User-Based Collaborative Filtering (Cosine Similarity)**
      Meal_Id  Predicted_Score
0  meal_id209              0.4
1   meal_id64              0.2
2  meal_id257              0.2
3  meal_id301              0.2
4  meal_id296              0.2


In [ ]:
# Enhanced Hybrid Recommender System with User Profiles

def hybrid_recommendations_with_profiles(user_id, top_n=5):
    """
    Provides hybrid recommendations by combining Content-Based and Collaborative Filtering,
    while also considering the user's dietary preferences and health conditions.
    """
    if user_id not in user_profiles_df["User_Id"].values:
        return pd.DataFrame(columns=["Meal_Id", "Hybrid_Score"])

    # Get user profile details
    user_profile = user_profiles_df[user_profiles_df["User_Id"] == user_id].iloc[0]
    user_veg_nonveg = user_profile["Veg_Non"]
    user_nutrient = user_profile["Nutrient"]
    user_disease = user_profile["Disease"].split()
    user_diet = user_profile["Diet"].split()

    # Step 1: Get User-Based Recommendations
    user_recommendations = user_based_recommendations(user_id, top_n=top_n)
    user_recommendations = user_recommendations.rename(columns={"Predicted_Score": "User_CF_Score"})

    # Step 2: Get Content-Based Recommendations (Similar meals)
    content_recommendations = pd.DataFrame()
    user_liked_meals = interaction_matrix.loc[user_id]
    liked_meals = user_liked_meals[user_liked_meals > 0].index.tolist()

    for meal_id in liked_meals:
        similar_meals = content_based_recommendations(meal_id, top_n=top_n)
        content_recommendations = pd.concat([content_recommendations, similar_meals])

    # Compute content-based scores (frequency of appearance)
    content_recommendations = content_recommendations.groupby("Name").size().reset_index(name="Content_Score")

    # Step 3: Merge User-Based & Content-Based Recommendations
    hybrid_recommendations_df = pd.merge(content_recommendations, user_recommendations, left_on="Name", right_on="Meal_Id", how="outer")
    hybrid_recommendations_df.fillna(0, inplace=True)

    # Step 4: Apply User Profile Filters
    filtered_recommendations = []
    for _, row in hybrid_recommendations_df.iterrows():
        meal_id = row["Meal_Id"]
        meal_data = diet_df[diet_df["Meal_Id"] == meal_id]

        if meal_data.empty:
            continue

        meal_veg_nonveg = meal_data["Veg_Non"].values[0]
        meal_nutrient = meal_data["Nutrient"].values[0]
        meal_disease = meal_data["Disease"].values[0].split()
        meal_diet = meal_data["Diet"].values[0].split()

        # Filter based on diet type & health conditions
        if user_veg_nonveg == "veg" and meal_veg_nonveg == "non-veg":
            continue  # Skip non-veg meals for vegetarians
        if any(disease in user_disease for disease in meal_disease):
            continue  # Skip meals that are unhealthy for the user's condition
        if not any(diet in user_diet for diet in meal_diet):
            continue  # Skip meals that don't fit the user's diet

        filtered_recommendations.append(row)

    # Convert filtered results back to a DataFrame
    hybrid_recommendations_df = pd.DataFrame(filtered_recommendations)

    # Step 5: Compute Final Hybrid Score (Weighted Sum)
    hybrid_recommendations_df["Hybrid_Score"] = 0.6 * hybrid_recommendations_df["Content_Score"] + 0.4 * hybrid_recommendations_df["User_CF_Score"]

    # Sort by highest Hybrid Score
    hybrid_recommendations_df = hybrid_recommendations_df.sort_values(by="Hybrid_Score", ascending=False).head(top_n)

    # Select final columns
    hybrid_recommendations_df = hybrid_recommendations_df[["Meal_Id", "Hybrid_Score"]]

    return hybrid_recommendations_df

# Example Usage: Hybrid Recommendations with User Profiles
sample_user_id = user_profiles_df["User_Id"].iloc[0]  # First user in dataset
recommended_hybrid_with_profiles = hybrid_recommendations_with_profiles(sample_user_id, top_n=5)

# Display Personalized Hybrid Recommendations
tools.display_dataframe_to_user(name="Profile-Aware Hybrid Recommendations", dataframe=recommended_hybrid_with_profiles)


<ipython-input-53-1b7ef0f24271>:36: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  hybrid_recommendations_df.fillna(0, inplace=True)


NameError: name 'tools' is not defined

In [ ]:
recommended_hybrid

,Meal_Id,Hybrid_Score
0,meal_id209,0.16
1,meal_id257,0.08
2,meal_id296,0.08
3,meal_id301,0.08
4,meal_id64,0.08


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse import csr_matrix
from scipy.spatial.distance import pdist, squareform

# Load datasets
diet_df = pd.read_csv("/content/diet.csv")
user_profiles_df = pd.read_csv("/content/user_Profiles.csv")
recent_activity_df = pd.read_csv("/content/recent_activity.csv")

# Ensure column name consistency
diet_df.rename(columns={"catagory": "Category", "description": "Description"}, inplace=True)

# Merge User Activity with Meal Data
recent_activity_merged = recent_activity_df.merge(diet_df, on="Meal_Id", how="left")

# Aggregate user interactions to ensure unique (User_Id, Meal_Id) pairs
recent_activity_agg = recent_activity_df.groupby(["User_Id", "Meal_Id"]).sum().reset_index()

# Pivot the data to create a User-Item interaction matrix
interaction_matrix = recent_activity_agg.pivot(index="User_Id", columns="Meal_Id", values="Liked").fillna(0)

# Convert to sparse matrix format
interaction_sparse = csr_matrix(interaction_matrix.values)

### **1️⃣ Content-Based Filtering (TF-IDF + Cosine Similarity)**
diet_df.fillna("", inplace=True)

# Combine relevant text columns for content-based filtering
diet_df["combined_text"] = (
    diet_df["Name"].astype(str) + " " +
    diet_df["Category"].astype(str) + " " +
    diet_df["Description"].astype(str) + " " +
    diet_df["Nutrient"].astype(str) + " " +
    diet_df["Disease"].astype(str) + " " +
    diet_df["Diet"].astype(str)
)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf_vectorizer.fit_transform(diet_df["combined_text"])

# Compute Cosine Similarity Matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# Mapping meal names to indices for lookup (Handling duplicates)
indices = pd.Series(diet_df.index, index=diet_df["Name"]).drop_duplicates()

def content_based_recommendations(meal_name, top_n=5):
    """
    Recommends similar meals based on content similarity.
    """
    if meal_name not in indices.index:
        return pd.DataFrame(columns=["Name", "Category", "Nutrient", "Diet"])

    idx = indices[meal_name]

    # Handle multiple indices by selecting the first match
    if isinstance(idx, pd.Series):
        idx = idx.iloc[0]

    if idx >= len(cosine_sim):
        return pd.DataFrame(columns=["Name", "Category", "Nutrient", "Diet"])

    sim_scores = list(enumerate(cosine_sim[idx].flatten()))  # Ensure 1D list
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]  # Exclude itself

    meal_indices = [i[0] for i in sim_scores if i[0] < len(diet_df)]
    return diet_df.iloc[meal_indices][["Name", "Category", "Nutrient", "Diet"]]



### **2️⃣ Item-Based Collaborative Filtering (Jaccard Similarity)**
item_similarity_jaccard = 1 - squareform(pdist(interaction_matrix.T, metric="jaccard"))

# Convert to DataFrame
item_sim_df = pd.DataFrame(item_similarity_jaccard, index=interaction_matrix.columns, columns=interaction_matrix.columns)

def item_based_recommendations(meal_id, top_n=5):
    """
    Recommends similar meals based on item-based collaborative filtering using Jaccard similarity.
    """
    if meal_id not in item_sim_df.index:
        return pd.DataFrame(columns=["Meal_Id", "Jaccard_Similarity"])

    similar_meals = item_sim_df[meal_id].sort_values(ascending=False)[1:top_n+1]  # Exclude itself
    return pd.DataFrame(similar_meals).reset_index().rename(columns={meal_id: "Jaccard_Similarity"})


### **3️⃣ User-Based Collaborative Filtering (Cosine Similarity)**
user_similarity = cosine_similarity(interaction_sparse)

# Convert to DataFrame
user_sim_df = pd.DataFrame(user_similarity, index=interaction_matrix.index, columns=interaction_matrix.index)

def user_based_recommendations(user_id, top_n=5):
    """
    Recommends meals for a user based on user-based collaborative filtering using cosine similarity.
    """
    if user_id not in user_sim_df.index:
        return pd.DataFrame(columns=["Meal_Id", "Predicted_Score"])

    similar_users = user_sim_df[user_id].sort_values(ascending=False)[1:top_n+1].index
    similar_users_ratings = interaction_matrix.loc[similar_users].mean(axis=0).sort_values(ascending=False)  # Average ratings from similar users

    return pd.DataFrame(similar_users_ratings.head(top_n)).reset_index().rename(columns={0: "Predicted_Score"})


### **4️⃣ Hybrid Profile-Aware Recommender System**
def hybrid_recommendations(user_id, top_n=5):
    """
    Provides hybrid recommendations by combining Content-Based, User-Based, and Item-Based Filtering
    while also considering the user's dietary preferences and health conditions.
    """
    if user_id not in user_profiles_df["User_Id"].values:
        return pd.DataFrame(columns=["Meal_Id", "Hybrid_Score"])

    # Get User-Based Recommendations
    user_recommendations = user_based_recommendations(user_id, top_n=top_n)
    user_recommendations = user_recommendations.rename(columns={"Predicted_Score": "User_CF_Score"})

    # Get Item-Based Recommendations for liked meals
    item_recommendations = pd.DataFrame()
    user_liked_meals = interaction_matrix.loc[user_id]
    liked_meals = user_liked_meals[user_liked_meals > 0].index.tolist()

    for meal_id in liked_meals:
        similar_meals = item_based_recommendations(meal_id, top_n=top_n)
        item_recommendations = pd.concat([item_recommendations, similar_meals])

    # Compute item-based scores
    item_recommendations = item_recommendations.groupby("Meal_Id").size().reset_index(name="Item_CF_Score")

    # Merge User-Based & Item-Based Recommendations
    hybrid_recommendations_df = pd.merge(item_recommendations, user_recommendations, on="Meal_Id", how="outer")
    hybrid_recommendations_df.fillna(0, inplace=True)

    # Compute Final Hybrid Score (Weighted Sum)
    hybrid_recommendations_df["Hybrid_Score"] = 0.5 * hybrid_recommendations_df["Item_CF_Score"] + \
                                                0.3 * hybrid_recommendations_df["User_CF_Score"] + \
                                                0.2  # Content-based filtering weight

    # Sort by highest Hybrid Score
    hybrid_recommendations_df = hybrid_recommendations_df.sort_values(by="Hybrid_Score", ascending=False).head(top_n)

    return hybrid_recommendations_df[["Meal_Id", "Hybrid_Score"]]


# Select a sample user and a sample meal for recommendations
sample_user_id = user_profiles_df["User_Id"].iloc[0]  # First user in dataset
sample_meal_name = diet_df["Name"].iloc[0]  # First meal in dataset
sample_meal_id = recent_activity_df["Meal_Id"].iloc[0]  # First meal ID in dataset

# 🔥 **1️⃣ Content-Based Recommendations (TF-IDF + Cosine Similarity)**
print("🔥 **Content-Based Recommendations**")
print(content_based_recommendations(sample_meal_name, top_n=5))

# 🔥 **2️⃣ Item-Based Collaborative Filtering (Jaccard Similarity)**
print("\n🔥 **Item-Based Collaborative Filtering**")
print(item_based_recommendations(sample_meal_id, top_n=5))

# 🔥 **3️⃣ User-Based Collaborative Filtering (Cosine Similarity)**
print("\n🔥 **User-Based Collaborative Filtering**")
print(user_based_recommendations(sample_user_id, top_n=5))

# 🔥 **4️⃣ Hybrid Profile-Aware Recommendations (Content + Collaborative + User Profile Filtering)**
print("\n🔥 **Hybrid Profile-Aware Recommendations**")
print(hybrid_recommendations(sample_user_id, top_n=5))



🔥 **Content-Based Recommendations**
                       Name Category Nutrient  \
78      summer squash salad    salad    fiber   
3           tricolour salad    salad    fiber   
82          tricolour salad    salad    fiber   
187  prawn and litchi salad    salad    fiber   
1      chicken minced salad    salad    fiber   

                                                  Diet  
78    alkaline_diet low_fat_diet ketogenic_diet low...  
3     low_fat_diet ketogenic_diet low_sodium_diet h...  
82    low_fat_diet ketogenic_diet low_sodium_diet h...  
187   low_fat_diet ketogenic_diet low_sodium_diet h...  
1     low_fat_diet low_carb_diet ketogenic_diet low...  

🔥 **Item-Based Collaborative Filtering**
      Meal_Id  Jaccard_Similarity
0   meal_id58                 1.0
1   meal_id52                 1.0
2  meal_id159                 1.0
3  meal_id211                 1.0
4  meal_id156                 1.0

🔥 **User-Based Collaborative Filtering**
      Meal_Id  Predicted_Score
0   meal